In [80]:
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics
import pandas as pd
import autosklearn.classification
import os
import featuretools as ft
import warnings
import autofeat 
from tpot import TPOTClassifier
import json 
import time


warnings.filterwarnings('ignore')

In [140]:
def run_as(X, y, target_ft, time_budget=30, include_preprocessors = None):
    try:
        os.remove('/tmp/autosklearn_regression_example_tmp')
        os.remove('/tmp/autosklearn_regression_example_out')
    except:
        pass
    #X = df.drop(columns=target_ft)
    #y = df[target_ft]
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=1)
    automl = autosklearn.classification.AutoSklearnClassifier(
        time_left_for_this_task=time_budget,
        per_run_time_limit=30,
        tmp_folder='./tmp/autosklearn_regression_example_tmp',
        output_folder='./tmp/autosklearn_regression_example_out',
        include_preprocessors = include_preprocessors
    )
    automl.fit(X_train, y_train)
    y_hat = automl.predict(X_test)
    
    acc = sklearn.metrics.accuracy_score(y_test, y_hat)
    f1_s = sklearn.metrics.f1_score(y_test, y_hat, average='macro')
    metrs = []
    metrs.append("Accuracy score - " + str(acc))
    metrs.append("F1 score - " + str(f1_s))
    
    res = ["","","","",f1_s,acc,"",automl.show_models()]
    
    
    return str(metrs),res

def run_tpot(X,y, target_ft,time_budget=30, include_preprocessors=None ):
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=1)
    pipeline_optimizer = TPOTClassifier(max_time_mins = time_budget/60, generations=None)
    pipeline_optimizer.fit(X_train, y_train)
    y_hat = pipeline_optimizer.predict(X_test)
    metrs = []
    metrs.append("Accuracy score - " + str(sklearn.metrics.accuracy_score(y_test, y_hat)))
    metrs.append("F1 score - " + str(sklearn.metrics.f1_score(y_test, y_hat, average='macro')))
    return str(metrs)

    
def gen_feats_featools(df):
    es = ft.EntitySet(id = 'df')
    es.entity_from_dataframe(entity_id = 'data', dataframe = df, 
                         make_index = True, index = 'index')
    feature_matrix, feature_defs = ft.dfs(entityset = es, target_entity = 'data', verbose = 1)
                                      #agg_primitives=["mean", "max", "min", "std", "skew"],
                                     # trans_primitives = ['add_numeric', 'multiply_numeric'])
    return feature_matrix

def gen_feats_autofeat(X,y):
    fsel = autofeat.FeatureSelector(verbose=1)
    X = fsel.fit_transform(X,y)
    return X
    
    
def run_test(df_path,target_ft, mode = 0, time_budget=30):
    results = []
    df = pd.read_csv(df_path)
    X = df.drop(columns=target_ft)
    y = df[target_ft]
    res_df = []
    autofeat_time = 0
    
    if mode == 0 or mode == 1:
        start = time.monotonic()
        X_new = gen_feats_autofeat(X,y)
        end = time.monotonic()
        autofeat_time = int(end-start)
        print("Autofeat_time: ",autofeat_time)
        rs = run_as(X_new,y,target_ft,time_budget=time_budget, include_preprocessors =["no_preprocessing"])
        results.append("Autosk Only with Preprocessing: " + rs[0])
        rs[1][0] = df_path[5:-4]
        rs[1][1] = str(time_budget/60)+'m'
        rs[1][2] = "Autofeat"
        rs[1][3] = "AutoSK"
        rs[1][6] = str(X_new.shape)
        res_df.append(rs[1])
    

    if mode == 0 or mode == 2:
        rs = run_as(X,y,target_ft, time_budget=time_budget+autofeat_time, include_preprocessors=None)   
        results.append("Autosk Only with Preprocessing: " + rs[0])
        rs[1][0] = df_path[5:-4]
        rs[1][1] = str(round((time_budget+autofeat_time)/60,2))+'m'
        rs[1][2] = "AutoSK"
        rs[1][3] = "AutoSK"
        rs[1][6] = str(X.shape)
        res_df.append(rs[1])
    if mode == 0 or mode == 3:
        rs = run_as(X,y,target_ft,time_budget=time_budget, include_preprocessors =["no_preprocessing"])
        results.append("Autosk Only without Preprocessing: " + rs[0])
        rs[1][0] = df_path[5:-4]
        rs[1][1] = str(round((time_budget+autofeat_time)/60,2))+'m'
        rs[1][2] = "None"
        rs[1][3] = "AutoSK"
        rs[1][6] = str(X.shape)
        res_df.append(rs[1])
    
    
    
    
    #if mode == 0 or mode == 4:
    #    X_new = gen_feats_featools(X)
    #    rs = run_as(X_new,y,target_ft,time_budget=time_budget, include_preprocessors =["no_preprocessing"])
    #    results.append("Autosk with Featuretools: " + rs)

    #if mode ==0 or mode == 5:
    #    rs = run_tpot(X,y,target_ft, time_budget=time_budget, include_preprocessors=None)   
    #    results.append("TPOT Only with Preprocessing: " + rs)
    
    
        
    print("===================================")
    print("Time budeget: ",time_budget)
    [print(x) for x in results]
    
    res_df =  pd.DataFrame(res_df, columns = ["Dataset","Time","Preprocessing","AutoML","Accuracy","F1","Shape","PipeLine"])
    res_df.drop(columns=["PipeLine"]).to_csv("results/"+df_path[5:])
    res_df.to_csv("results/pipe_"+df_path[5:])
    
    return res_df

In [141]:
df_path = "data/gina.csv"
target_ft = "class"
res = run_test(df_path, target_ft, mode=0 ,time_budget=30)

[featsel] Scaling data...done.
[featsel] Feature selection run 1/5
[featsel] Feature selection run 2/5
[featsel] Feature selection run 3/5
[featsel] Feature selection run 4/5
[featsel] Feature selection run 5/5
[featsel] 112 features after 5 feature selection runs
[featsel] 112 features after correlation filtering
[featsel] 59 features after noise filtering
Autofeat_time:  45
[WARNING] [2020-07-29 12:40:44,308:AutoML(1):d01c2652e5f4de11f6acdd88727a25ce] Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (29.717132)
[WARNING] [2020-07-29 12:40:44,312:AutoML(1):d01c2652e5f4de11f6acdd88727a25ce] Capping the per_run_time_limit to 14.0 to have time for a least 2 models in each process.
[WARNING] [2020-07-29 12:42:19,423:AutoML(1):5193c1c86d0dd70aa3b0a1499d7fd4e0] Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (29.628175)
[WARNING] [202

In [142]:
res

,Dataset,Time,Preprocessing,AutoML,Accuracy,F1,Shape,PipeLine
0,gina,0.5m,Autofeat,AutoSK,0.928914,0.929024,"(3153, 59)","[(0.320000, SimpleClassificationPipeline({'bal..."
1,gina,1.25m,AutoSK,AutoSK,0.935280,0.935361,"(3153, 970)","[(0.820000, SimpleClassificationPipeline({'bal..."
2,gina,1.25m,None,AutoSK,0.912299,0.912548,"(3153, 970)","[(0.600000, SimpleClassificationPipeline({'bal..."


In [19]:
#!rm -r tmp
#df = pd.read_csv("blood.csv")
#target_ft = "class"
#run_test(df, target_ft, mode=2, time_budget=60)

In [20]:
#df = pd.read_csv("winequality-red.csv")
#target_ft = "quality"
#run_test(df, target_ft, mode=2 ,time_budget=30)

In [21]:
#df = pd.read_csv("data/airlines.csv")

In [22]:
#!rm -r tmp
#df = pd.read_csv("data/airlines.csv").drop(columns=["Airline","AirportFrom","AirportTo"])
#target_ft = "Delay"
#run_test(df, target_ft, mode=2 ,time_budget=30)

In [23]:
#!rm -r tmp
#df = pd.read_csv("data/gina.csv")
#target_ft = "class"
#run_test(df, target_ft, mode=2 ,time_budget=30)

In [24]:
#!rm -r tmp
#df = pd.read_csv("data/gina.csv")
#target_ft = "class"
#run_test(df, target_ft, mode=4 ,time_budget=420)